In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [2]:
# Load the dataset using Pandas
data = pd.read_csv('diabetes.csv')

In [3]:
# For x: Extract out the dataset from all the rows (all samples) and all columns except last column (all features). 
# For y: Extract out the last column (which is the label)
# Convert both to numpy using the .values method
x = data.iloc[:,0:-1].values
y_string = list(data.iloc[:,-1])

In [4]:
print(x[:3])
print(y_string[:3])

[[  6.  148.   72.   35.    0.   33.6  50. ]
 [  1.   85.   66.   29.    0.   26.6  31. ]
 [  8.  183.   64.    0.    0.   23.3  32. ]]
['positive', 'negative', 'positive']


In [5]:
# neural network only understand numbers! So convert the string to labels
y_int = []
for s in y_string:
    if s == 'positive':
        y_int.append(1)
    else:
        y_int.append(0)

In [6]:
# convert to an array
y = np.array(y_int, dtype='float64')

In [7]:
# Feature Normalization. All features should have the same range of values (-1,1)
sc = StandardScaler()
x = sc.fit_transform(x)

In [8]:
#  convert the arrays to PyTorch tensors
x = torch.tensor(x)
# add an extra dimension to convert this array to 2D
y = torch.tensor(y).unsqueeze(1)

In [9]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [10]:
class Dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        # Get one item from the dataset
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

In [11]:
dataset = Dataset(x,y)

In [12]:
len(dataset)

768

In [13]:
# Load the data to dataloader for batch processing and shuffling
train_loader = torch.utils.data.DataLoader(dataset = dataset,
                           batch_size = 32,
                           shuffle = True)

In [14]:
train_loader

In [15]:
# print data loader
print("There is {} batches in the dataset".format(len(train_loader)))
for (x,y) in train_loader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break

There is 24 batches in the dataset
For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


In [16]:
class Model(nn.Module):
    def __init__(self,input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [19]:
# Create the network
net = Model(7,1)
# define loss function
criterion = torch.nn.BCELoss(reduction='mean')
# use SGD with momentum with a learning rate of 0.1
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

C:\Users\najee\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [29]:
# Train the network 
num_epochs = 200
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # Feed Forward
        output = net(inputs)
        # Loss Calculation
        loss = criterion(output, labels)
         # Clear the gradient buffer
        optimizer.zero_grad()
        # Backpropagation 
        loss.backward()
        # Weight Update: w <-- w - lr * gradient
        optimizer.step()
    
    #Accuracy
    # Since we are using a sigmoid, we will need to perform some thresholding
    output = (output>0.5).float()
    # Accuracy: (output == labels).float().sum() / output.shape[0]
    accuracy = (output == labels).float().mean()
    # Print statistics 
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {round(loss.item(),3)}, Accuracy: {round(accuracy.item(),3)}")

Epoch 1/200, Loss: 0.45, Accuracy: 0.719
Epoch 2/200, Loss: 0.323, Accuracy: 0.812
Epoch 3/200, Loss: 0.374, Accuracy: 0.875
Epoch 4/200, Loss: 0.438, Accuracy: 0.812
Epoch 5/200, Loss: 0.191, Accuracy: 0.938
Epoch 6/200, Loss: 0.267, Accuracy: 0.906
Epoch 7/200, Loss: 0.411, Accuracy: 0.688
Epoch 8/200, Loss: 0.342, Accuracy: 0.844
Epoch 9/200, Loss: 0.467, Accuracy: 0.781
Epoch 10/200, Loss: 0.457, Accuracy: 0.719
Epoch 11/200, Loss: 0.411, Accuracy: 0.812
Epoch 12/200, Loss: 0.464, Accuracy: 0.75
Epoch 13/200, Loss: 0.511, Accuracy: 0.781
Epoch 14/200, Loss: 0.431, Accuracy: 0.781
Epoch 15/200, Loss: 0.621, Accuracy: 0.688
Epoch 16/200, Loss: 0.496, Accuracy: 0.812
Epoch 17/200, Loss: 0.566, Accuracy: 0.75
Epoch 18/200, Loss: 0.214, Accuracy: 0.938
Epoch 19/200, Loss: 0.342, Accuracy: 0.781
Epoch 20/200, Loss: 0.39, Accuracy: 0.719
Epoch 21/200, Loss: 0.477, Accuracy: 0.75
Epoch 22/200, Loss: 0.484, Accuracy: 0.75
Epoch 23/200, Loss: 0.265, Accuracy: 0.906
Epoch 24/200, Loss: 0.766,